In [1]:
import pandas as pd
import string
from pprint import pprint
from time import time
from nltk.corpus import stopwords

In [2]:
#Get the spam data collection 
df= pd.read_csv("SpamCollection", sep='\t',names=['response','message'])

In [3]:
df.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV


ModuleNotFoundError: No module named 'sklearn.grid_search'

In [5]:
df.describe()


,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [6]:
df.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [7]:
#Verify length of the messages and also add it as a new column 
df['length'] = df['message'].apply(len)

In [8]:
df.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [9]:
#define a function to get rid of stopwords present in the messages
def message_text_process(mess):
    no_punctuation=[char for char in mess if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return[words for words in no_punctuation.split() if words.lower() not in stopwords.words('english')]

In [10]:

df['message'].head(5).apply(message_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [11]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
#use bag of words by applying the function and fit the data into it
bagofwords= CountVectorizer(analyzer=message_text_process).fit(df['message'])

In [13]:
#print length of bag of words stored in the vocabulary_ attribute
print(len(bagofwords.vocabulary_))

11425


In [15]:
message_bagofwords= bagofwords.transform(df['message'])


In [16]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(message_bagofwords)

In [18]:
#print shape of the tfidf 
messag_tfidf=tfidf_transformer.transform(message_bagofwords)
print(message_bagofwords.shape)

(5572, 11425)


In [20]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spamdetect_model=MultinomialNB().fit(messag_tfidf,df['response'])

In [28]:
#check model for the predicted and expected value say for message#2 and message#5
message=df['message'][4]
bagofwords_for_message = bagofwords.transform(['message'])
tfidf=tfidf_transformer.transform(bagofwords_for_message)

In [30]:
print('predicted', spamdetect_model.predict(tfidf)[0])
print('expected',df.response[4])

predicted ham
expected ham
